In [ ]:
%pip install pandas
%pip install google.colab
%pip install requests

In [ ]:
import pandas as pd
import requests
import time
import re

group_id = -29534144
post_id1 = 21176327
post_id2 = 21171752

def clean_string(input_string: str) -> str:
    cleaned_string = re.sub(r'\[.*?\]', '', input_string)
    cleaned_string = re.sub(r',', '', cleaned_string)
    cleaned_string = cleaned_string.strip()
    return cleaned_string

token = ''
version = 5.131
def GetCommentsBranch(owner_id, post_id, count, start_comment_id = 0, comment_id = 0):
    return requests.get('https://api.vk.com/method/wall.getComments',
                        params = {
                            'v' : version,
                            'owner_id' : owner_id,
                            'post_id' : post_id,
                            'count' : count,
                            'start_comment_id' : start_comment_id,
                            'comment_id' : comment_id
                        },
                        headers={"Authorization": f"Bearer {token}"})

def get_comments(owner_id: int, post_id: int) -> list[str]:
    COMMS = []
    IntPoint = GetCommentsBranch(owner_id, post_id, 1).json().get('response',{})
    COUNT = IntPoint.get('current_level_count')
    COMMENT_ID = IntPoint.get('items')[0].get('id')
    i = 1
    while COUNT > 0:
        res = GetCommentsBranch(owner_id, post_id, 100 if COUNT > 100 else COUNT, COMMENT_ID).json().get('response',{})
        for comment in res.get('items'):
            COMMS.append(comment.get('text'))
            SUB_COUNT = comment.get('thread').get('count')
            SUB_COMMENT_ID = comment.get('id')
            si = 1
            while SUB_COUNT > 0:
                current = 0
                subres = GetCommentsBranch(owner_id, post_id,100 if SUB_COUNT > 100 else SUB_COUNT, current,SUB_COMMENT_ID).json()
                if not subres.get('response'):
                    print(subres)
                subres = subres.get('response')
                for subcomment in subres.get('items') if subres else []:
                    COMMS.append(clean_string(subcomment.get('text')))
                    current = subcomment.get('id')
                    si+=1

                SUB_COUNT -=100
            time.sleep(0.25)
            i+=1
            COMMENT_ID = comment.get('id')
        COUNT-=100
    return COMMS

In [ ]:
test_comments_post1 = get_comments(group_id, post_id1)
test_comments_post2 = get_comments(group_id, post_id2)
comments_post1_df = pd.DataFrame({'comment': test_comments_post1})
comments_post2_df = pd.DataFrame({'comment': test_comments_post2})
comments_post1_df.to_csv('comments1.csv')
comments_post2_df.to_csv('comments2.csv')


comments_post1_df

In [ ]:
comments_post2_df